<a href="https://colab.research.google.com/github/erlichsefi/ScrapeAnything/blob/main/browser_base_translation%20/JavaScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# According to: https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.xrghamjPz2/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.7ExrPKavcE/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.dRTY7Z8sqQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [3]:
!apt install chromium-chromedriver
! pip install pandas

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser liblzo2-2 snapd squashfs-tools
Suggested packages:
  apparmor-profiles-extra apparmor-utils zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 snapd
  squashfs-tools
0 upgraded, 6 newly installed, 0 to remove and 52 not upgraded.
Need to get 38.6 MB of archives.
After this operation, 174 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 apparmor amd64 2.13.3-7ubuntu5.2 [502 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 liblzo2-2 amd64 2.10-2 [50.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 squashfs-tools amd64 1:4.4-1ubuntu0.3 [117 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 snapd amd64 2.58+20.04.1 [37.9 MB]
Get:5 http://ar

In [59]:
script_with_logs = """
// Get all elements in the HTML page
const elements = document.getElementsByTagName('*');

// Create an array to store the element details
const elementDetails = [];

// Iterate through each element
for (let i = 0; i < elements.length; i++) {
  const element = elements[i];

  // Get the bounding rectangle of the element
  const rect = element.getBoundingClientRect();

  // Get the text content of the element
  const textContent = element.textContent.trim();

  // Get the tooltip value if it exists
  const tooltip = element.hasAttribute('title') ? element.getAttribute('title') : '';

  // Store the element, its bounding rectangle, text content, and tooltip details
  const elementInfo = {
    element: element,
    rect: rect,
    textContent: textContent,
    ariaLabel: element.hasAttribute('aria-label') ? element.getAttribute('aria-label') : '',
    tooltip: tooltip,
    e_type: element.nodeName
  };
  if (elementInfo.rect.width > 0 && elementInfo.rect.height > 0){
  // Add the element details to the array
   if (elementInfo.tooltip != '' || elementInfo.textContent != '' || elementInfo.ariaLabel != ''){
    elementDetails.push(elementInfo);
    }
  }
}

let parents = elementDetails.map(e=>e.element.parentElement);
let withoutParents = elementDetails.filter(elementDetail=> !parents.includes(elementDetail.element));

// Display the element details
console.log("X,Y,Width,Height,ElementType,textContent,TooltipValue,AriaLabel");
console.log(withoutParents.map( e=> e.rect.x+","+e.rect.y+","+e.rect.width+","+e.rect.height+","+e.e_type+","+e.rect.textContent+","+e.tooltip+","+e.ariaLabel).join("\\n"));
"""

In [89]:
def screen_to_table(url):

  from selenium import webdriver
  from selenium.webdriver.chrome.service import Service
  import pandas as pd
  import io  
  import re


  service = Service(executable_path=r'/usr/bin/chromedriver')
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.headless = True
  wd = webdriver.Chrome(service=service,options=chrome_options)
  wd.get(url)

  file_name = re.sub(r'[^a-zA-Z0-9_-]', '_', url)
  file_name = f"{file_name}.png"
  wd.save_screenshot(file_name)
  #wd.title
  script = f"""
  var consoleLogs = [];
  var originalLog = console.log;
  console.log = function(message) {{
      consoleLogs.push(message);
      originalLog.apply(console, arguments);
  }};

  {script_with_logs}

  return consoleLogs;
  """

  logs = wd.execute_script(script)
  
  return pd.read_csv(io.StringIO("\n".join(logs)), sep=",")


screen_to_table('https://www.google.com')

<ipython-input-89-d9db703a8ded>:14: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


,X,Y,Width,Height,ElementType,textContent,TooltipValue,AriaLabel
0,21.000000,17,46.593750,26,A,undefined,NaN,NaN
1,77.593750,17,43.468750,26,A,undefined,NaN,NaN
2,523.406250,18,33.953125,24,A,undefined,NaN,Gmail (opens a new tab)
3,572.359375,18,42.640625,24,A,undefined,NaN,Search for Images (opens a new tab)
4,634.000000,10,40.000000,40,A,undefined,NaN,Google apps
5,711.718750,22,44.562500,16,SPAN,undefined,NaN,NaN
6,264.000000,60,272.000000,92,DIV,undefined,NaN,NaN
7,123.000000,179,33.000000,46,DIV,undefined,NaN,NaN
8,156.000000,190,443.000000,27,TEXTAREA,undefined,Search,Search
9,603.000000,179,40.000000,44,DIV,undefined,NaN,Search by voice


In [77]:
logs = wd.execute_script(script)
 
df = pd.read_csv(io.StringIO("\n".join(logs)), sep=",")
df

,X,Y,Width,Height,ElementType,textContent,TooltipValue,AriaLabel
0,21.000000,17,46.593750,26,A,undefined,NaN,NaN
1,77.593750,17,43.468750,26,A,undefined,NaN,NaN
2,523.406250,18,33.953125,24,A,undefined,NaN,Gmail (opens a new tab)
3,572.359375,18,42.640625,24,A,undefined,NaN,Search for Images (opens a new tab)
4,634.000000,10,40.000000,40,A,undefined,NaN,Google apps
5,711.718750,22,44.562500,16,SPAN,undefined,NaN,NaN
6,264.000000,60,272.000000,92,DIV,undefined,NaN,NaN
7,123.000000,179,33.000000,46,DIV,undefined,NaN,NaN
8,156.000000,190,443.000000,27,TEXTAREA,undefined,Search,Search
9,603.000000,179,40.000000,44,DIV,undefined,NaN,Search by voice


In [ ]:
# Here we will ask the LLM

In [73]:
from selenium.webdriver.common.action_chains import ActionChains
# Get X and Y coordinates
x = 523.406250
y = 18

# Perform mouse click at X and Y coordinates
actions = ActionChains(wd)
actions.move_by_offset(x, y)
actions.click()
actions.perform()

In [75]:
wd.

AttributeError: ignored

,X,Y,Width,Height,ElementType,textContent,TooltipValue,AriaLabel
0,21.000000,17,46.593750,26,A,undefined,NaN,NaN
1,77.593750,17,43.468750,26,A,undefined,NaN,NaN
2,523.406250,18,33.953125,24,A,undefined,NaN,Gmail (opens a new tab)
3,572.359375,18,42.640625,24,A,undefined,NaN,Search for Images (opens a new tab)
4,634.000000,10,40.000000,40,A,undefined,NaN,Google apps
5,711.718750,22,44.562500,16,SPAN,undefined,NaN,NaN
6,264.000000,60,272.000000,92,DIV,undefined,NaN,NaN
7,123.000000,179,33.000000,46,DIV,undefined,NaN,NaN
8,156.000000,190,443.000000,27,TEXTAREA,undefined,Search,Search
9,603.000000,179,40.000000,44,DIV,undefined,NaN,Search by voice


In [ ]:

on_screen_data = """
X,Y,Width,Height,ElementType,textContent,TooltipValue,AriaLabel
VM1301:41 464.40625,18,33.953125,24,A,undefined,,Gmail (opens a new tab)
513.359375,18,42.640625,24,A,undefined,,Search for Images (opens a new tab)
575,10,40,40,A,undefined,,Google apps
623,10,40,40,A,undefined,,Google Account: Sefi Erlich  
(erlichsefi@gmail.com)
202.5,258,272,92,DIV,undefined,,
61.5,377,33,46,DIV,undefined,,
94.5,388,443,27,TEXTAREA,undefined,Search,Search
541.5,377,40,44,DIV,undefined,,Search by voice
581.5,377,40,44,DIV,undefined,,Search by image
197.59375,453,127.40625,36,INPUT,undefined,,Google Search
336.890625,453,142.5,36,INPUT,undefined,,I'm Feeling Lucky
353.953125,516,38.109375,15,A,undefined,,
395.6875,516,34.09375,15,A,undefined,,
0,875,677,48,DIV,undefined,,
24.328125,923,66.59375,46,A,undefined,,
90.921875,923,99.265625,46,A,undefined,,
190.1875,923,86.8125,46,A,undefined,,
277,923,146.703125,46,A,undefined,,
428.03125,923,75.90625,46,A,undefined,,
503.9375,923,68.125,46,A,undefined,,
572.0625,923,80.59375,46,DIV,undefined,,
"""